# Project Report I: Data Cleaning

**Report Brief**<br>
In this progress report, I'm hoping to do as much of the data processing as I can, although I don't expect to finish. The breakdown of my project likely means that the heaviest proportion of work will fall on data cleaning/organization, and then EDA, with visualization and analysis being the least intensive section. This is because my project is product-centric--first and foremost, I'm trying to make a *thing* that can classify data.
___
**Goals**

Here's a list of things I'm hoping to accomplish in the first portion of my project:
1. ~Fix the Markdown formatting for my README and other files~
2. ~Add data sample to GitHub~
3. read in data from different folders/sources **unsuccessful**
4. create a sample DF with a few columns of useful data 
    1. text
    2. folder it is in
    3. length of email
    4. to/from
5. visualize some of the data with a few statistical tests or plots

B and D will likely be fairly difficult, and just reading in the data isn't easy, so I'm really aiming to accomplish 1, 2, 3, and 4. Lets see how it goes!
___
**Sharing Data**<br>
In regards to sharing data, there is a bit of a problem in that the source from which I took my data (http://www.cs.cmu.edu/~enron/) doesn't make mention of any type of licensing. I'm going to need to look into this further, but I'm tentatively planning on my data sample being just one (anonymized) person's emails, since this is the most natural data member of the corpus. This will clearly demonstrate the shape and scope of the corpus without taking up too much space or compromising privacy.

**Reading In Data**

imports, setting up file path

In [1]:
import nltk
from nltk.corpus import PlaintextCorpusReader as cr
import pandas as pd
import numpy as np

In [2]:
corpus_root = '../../../../../Enron_Emails/maildir/'
all_allen_p = 'allen-p/all_documents'
all_files = cr(corpus_root + all_allen_p, ".*")
print(len(all_files.fileids()))
#needs a way to go one in, regardless of name
#plaintext reader?

628


prepping email for NotePad-level readability

In [3]:
text = ""
file = open(corpus_root + all_allen_p + "/1")
for line in file:
    text += line
file.close()
email = text.split('\n')

In [4]:
email

['Message-ID: <29790972.1075855665306.JavaMail.evans@thyme>',
 'Date: Wed, 13 Dec 2000 18:41:00 -0800 (PST)',
 'From: 1.11913372.-2@multexinvestornetwork.com',
 'To: pallen@enron.com',
 "Subject: December 14, 2000 - Bear Stearns' predictions for telecom in Latin",
 ' America',
 'Mime-Version: 1.0',
 'Content-Type: text/plain; charset=us-ascii',
 'Content-Transfer-Encoding: 7bit',
 'X-From: Multex Investor <1.11913372.-2@multexinvestornetwork.com>',
 'X-To: <pallen@enron.com>',
 'X-cc: ',
 'X-bcc: ',
 'X-Folder: \\Phillip_Allen_Dec2000\\Notes Folders\\All documents',
 'X-Origin: Allen-P',
 'X-FileName: pallen.nsf',
 '',
 "In today's Daily Update you'll find free reports on",
 'America Online (AOL), Divine Interventures (DVIN),',
 'and 3M (MMM); reports on the broadband space, Latin',
 'American telecom, and more.',
 '',
 "For free research, editor's picks, and more come to the Daily Investor:",
 'http://www.multexinvestor.com/AF004627/magazinecover.asp?promo=unl&d=20001214#',
 'investor

HUGE problem. any categories on multiple lines are split too. no good!

**my fix**: lets loop through the list, and if we find an element in the list that starts with whitespace, we append it to the previous element.

In [5]:
count = 0
for line in email:
    if line == '':
        pass
    else:
        if line[0].isspace():
            email[count-1] += line
            del email[count]
            print(count)
    count += 1
email
#this is significantly better! we now have our completed subject 
#we simply need to remove the extra line under the governing category (Subject)

5


['Message-ID: <29790972.1075855665306.JavaMail.evans@thyme>',
 'Date: Wed, 13 Dec 2000 18:41:00 -0800 (PST)',
 'From: 1.11913372.-2@multexinvestornetwork.com',
 'To: pallen@enron.com',
 "Subject: December 14, 2000 - Bear Stearns' predictions for telecom in Latin America",
 'Mime-Version: 1.0',
 'Content-Type: text/plain; charset=us-ascii',
 'Content-Transfer-Encoding: 7bit',
 'X-From: Multex Investor <1.11913372.-2@multexinvestornetwork.com>',
 'X-To: <pallen@enron.com>',
 'X-cc: ',
 'X-bcc: ',
 'X-Folder: \\Phillip_Allen_Dec2000\\Notes Folders\\All documents',
 'X-Origin: Allen-P',
 'X-FileName: pallen.nsf',
 '',
 "In today's Daily Update you'll find free reports on",
 'America Online (AOL), Divine Interventures (DVIN),',
 'and 3M (MMM); reports on the broadband space, Latin',
 'American telecom, and more.',
 '',
 "For free research, editor's picks, and more come to the Daily Investor:",
 'http://www.multexinvestor.com/AF004627/magazinecover.asp?promo=unl&d=20001214#',
 'investor',
 '

**that didn't work**<br>
so we've got a problem on our hands. I've spent an hour or so on this and need to move on and come back later.

In [16]:
MessageID_tup = email[0].split(":")
Date_tup = email[1].split(":")
From_tup = email[2].split(":")
To_tup = email[3].split(":")
Subject_tup = email[4].split(":") 
MimeVers_tup = email[5].split(":")
ContentType_tup = email[6].split(":")
Encoding_tup = email[7].split(":")
XFrom_tup = email[8].split(":")
XTo_tup = email[9].split(":")
CC_tup = email[10].split(":")
BCC_tup = email[11].split(":")
XFolder_tup = email[12].split(":")
XOrigin_tup = email[13].split(":")
XFilename_tup = email[14].split(":")
header = [MessageID_tup, Date_tup, From_tup, To_tup, Subject_tup, MimeVers_tup, ContentType_tup, Encoding_tup, 
         XFrom_tup, XTo_tup, CC_tup, BCC_tup, XFolder_tup, XOrigin_tup, XFilename_tup]

In [17]:
for cat in header:
    print(cat)

['Message-ID', ' <29790972.1075855665306.JavaMail.evans@thyme>']
['Date', ' Wed, 13 Dec 2000 18', '41', '00 -0800 (PST)']
['From', ' 1.11913372.-2@multexinvestornetwork.com']
['To', ' pallen@enron.com']
['Subject', " December 14, 2000 - Bear Stearns' predictions for telecom in Latin America"]
['Content-Type', ' text/plain; charset=us-ascii']
['Content-Transfer-Encoding', ' 7bit']
['X-From', ' Multex Investor <1.11913372.-2@multexinvestornetwork.com>']
['X-To', ' <pallen@enron.com>']
['X-cc', ' ']
['X-bcc', ' ']
['X-Folder', ' \\Phillip_Allen_Dec2000\\Notes Folders\\All documents']
['X-Origin', ' Allen-P']
['X-FileName', ' pallen.nsf']
['']


In [10]:
email[16:]

['America Online (AOL), Divine Interventures (DVIN),',
 'and 3M (MMM); reports on the broadband space, Latin',
 'American telecom, and more.',
 '',
 "For free research, editor's picks, and more come to the Daily Investor:",
 'http://www.multexinvestor.com/AF004627/magazinecover.asp?promo=unl&d=20001214#',
 'investor',
 '',
 '***************************************************************',
 'You are receiving this mail because you have registered for',
 'Multex Investor. To unsubscribe, see bottom of this message.',
 '***************************************************************',
 '',
 '======================== Sponsored by =========================',
 'Would you own just the energy stocks in the S&P 500?',
 'Select Sector SPDRs divides the S&P 500 into nine sector index funds.',
 'Pick and choose just the pieces of the S&P 500 you like best.',
 'http://www.spdrindex.com',
 '===============================================================',
 '',
 "Featured in today's edition of the D

now when we print the tuples, we get matching pairs. This will only be the case when I fix that algorithm (hard coded for now)

In [11]:
text = email[16:]
for line in text:
    if line == '':
        text.remove(line)
text

['America Online (AOL), Divine Interventures (DVIN),',
 'and 3M (MMM); reports on the broadband space, Latin',
 'American telecom, and more.',
 "For free research, editor's picks, and more come to the Daily Investor:",
 'http://www.multexinvestor.com/AF004627/magazinecover.asp?promo=unl&d=20001214#',
 'investor',
 '***************************************************************',
 'You are receiving this mail because you have registered for',
 'Multex Investor. To unsubscribe, see bottom of this message.',
 '***************************************************************',
 '======================== Sponsored by =========================',
 'Would you own just the energy stocks in the S&P 500?',
 'Select Sector SPDRs divides the S&P 500 into nine sector index funds.',
 'Pick and choose just the pieces of the S&P 500 you like best.',
 'http://www.spdrindex.com',
 '===============================================================',
 "Featured in today's edition of the Daily Update:",
 "1. 

In [12]:
textList = email[16:]
textStr = ""
for line in textList:
    textStr += line
textStr

'America Online (AOL), Divine Interventures (DVIN),and 3M (MMM); reports on the broadband space, LatinAmerican telecom, and more.For free research, editor\'s picks, and more come to the Daily Investor:http://www.multexinvestor.com/AF004627/magazinecover.asp?promo=unl&d=20001214#investor***************************************************************You are receiving this mail because you have registered forMultex Investor. To unsubscribe, see bottom of this message.***************************************************************======================== Sponsored by =========================Would you own just the energy stocks in the S&P 500?Select Sector SPDRs divides the S&P 500 into nine sector index funds.Pick and choose just the pieces of the S&P 500 you like best.http://www.spdrindex.com===============================================================Featured in today\'s edition of the Daily Update:1. SPECIAL ANNOUNCEMENT: Treat yourself to Multex Investor\'s NEW PersonalFinance Chann

huh. this is so weird. for some reason, if I only run the removes on this once, it removes *some* of the whitespace at the beginning, but misses at the end. very odd.<br>
**Problem**: So, it's going to take a lot of time and energy to wrangle this list into a nice string to put into the DF. In fact, I need to think a *lot* more about what version of this text I'll be storing in my panda. I'm not really sure what I should be parsing this text for... something to think about in the future.
___
at the very least, we now have a string that represents the text of the email. lets add it to the list of tuples now.

In [13]:
header.append(["Text",textStr])

In [14]:
for cat in header:
    print(cat)

['Message-ID', ' <29790972.1075855665306.JavaMail.evans@thyme>']
['Date', ' Wed, 13 Dec 2000 18', '41', '00 -0800 (PST)']
['From', ' 1.11913372.-2@multexinvestornetwork.com']
['To', ' pallen@enron.com']
['Subject', " December 14, 2000 - Bear Stearns' predictions for telecom in Latin America", 'C', 'o', 'n', 't', 'e', 'n', 't', '-', 'T', 'y', 'p', 'e', ':', ' ', 't', 'e', 'x', 't', '/', 'p', 'l', 'a', 'i', 'n', ';', ' ', 'c', 'h', 'a', 'r', 's', 'e', 't', '=', 'u', 's', '-', 'a', 's', 'c', 'i', 'i']
['Content-Transfer-Encoding', ' 7bit']
['X-From', ' Multex Investor <1.11913372.-2@multexinvestornetwork.com>']
['X-To', ' <pallen@enron.com>']
['X-cc', ' ']
['X-bcc', ' ']
['X-Folder', ' \\Phillip_Allen_Dec2000\\Notes Folders\\All documents']
['X-Origin', ' Allen-P']
['X-FileName', ' pallen.nsf']
['']
["In today's Daily Update you'll find free reports on"]
['Text', 'America Online (AOL), Divine Interventures (DVIN),and 3M (MMM); reports on the broadband space, LatinAmerican telecom, and mor

In [15]:
DF = pd.DataFrame()
for tup in header:
    for x,y in tup:
        print(x,y)

ValueError: too many values to unpack (expected 2)

this is going to be the error that does me in. onto the summary

## Project Report 1: Summary
**What Went Well**:<br>
1. Outside the immediate bounds of this notebook, I've definitely improved my ability to use Markdown. I definitely think my notebooks and README's look a lot better.
2. Searching through the directory. At the beginning of this notebook, I did test the opening and reading in of mqny different files from the directory. This may not be important *this minute*, but the ability to apply my cleaning, exploration, and analysis to all parts of this corpus cannot be understated.
3. I have a clear direction going forward. Truthfully, this notebook was a huge failure and I did not prepare for how difficult it would be to do simple tasks like reading in data. That being said, I have a much clearer idea of what I need moving forward into Spring Break.
___
**What Went Poorly**:
1. I really struggled in converting data types in the wild. Reading the data in, getting the data into a tuple, and transferring this data into a dataframe were all difficult/impossible for me in this notebook. See section below.
2. I wasn't thinking ahead. You could probably tell by 5-6 cells in that I wasn't going to make it for the long haul in this one. It's *incredibly important* to make sure when you are reading newly formatted data that you put it into the correct data type (namely, my problems with the tuples).
___
**What's Next**:
1. A list of things I need to fix from this notebook:
    1. A generalizable way to turn the heading section into a tuple with the category and the data. I was close here, but the emails with multiple lines per category (i.e. an email sent to 30 people) tripped me up here.
    2. A better text string. This is one I need to think about more. I'm definitely going to need something to put into the dataframe, as far as the body of the email is concerned. The question is, what should I use? My current version needs fixing.
    3. A DF. This one will come last, but I definitely need a DF that can be expanded to include N number of emails.
___ 
**Conclusion**:<br>
All in all, a very rough go of things. After midterms this week I'll be able to pour more time into this project, which is what I need. I definitely struggle with goal setting on individually driven projects, which is something I'll need to work on.